# Eat Safe, Love

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

## Part 1: Database and Jupyter Notebook Set Up

In [96]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint


In [97]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [98]:
# confirm that our new database was created
db_list = mongo.list_database_names()
print("Connected to MongoDB!")
print("Existing Databases:", db_list)

Connected to MongoDB!
Existing Databases: ['Autosaurus', 'admin', 'classDB', 'config', 'epa', 'fruitsdb', 'local', 'met', 'uk_food']


In [99]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [100]:
# review the collections in our new database
collections = db.list_collection_names()

In [101]:
# review the collections in our new database
print(f"Collections in 'uk_food':", collections)

Collections in 'uk_food': ['establishments']


In [102]:
# review a document in the establishments collection
collection = db['establishments']

document = collection.find_one()

# Print the document
pprint(document)

{'Address': '45 Baker Street, London',
 'BusinessName': 'Gourmet Delight',
 'Cuisine': 'Fine Dining',
 'LastInspection': '2025-02-25',
 'Postcode': 'W1U 8EW',
 'Rating': 4,
 '_id': ObjectId('67c0d835a1054d82bb6e3089')}


In [103]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [104]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [105]:
# Insert the new restaurant into the collection
inserted_id = db['establishments'].insert_one(penang_flavours).inserted_id

print(f"New restaurant inserted with _id: {inserted_id}")

New restaurant inserted with _id: 67c0dff2a1054d82bb6e3095


In [106]:
# Check that the new restaurant was inserted

inserted_document = db['establishments'].find_one({"_id": inserted_id})
print(inserted_document)

{'_id': ObjectId('67c0dff2a1054d82bb6e3095'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [107]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = db['establishments'].find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  
)

print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': ''}


In [108]:
print(db['establishments'].find_one())

{'_id': ObjectId('67c0d835a1054d82bb6e3089'), 'BusinessName': 'Gourmet Delight', 'Address': '45 Baker Street, London', 'Postcode': 'W1U 8EW', 'Rating': 4, 'Cuisine': 'Fine Dining', 'LastInspection': '2025-02-25'}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [109]:
# Update the new restaurant with the correct BusinessTypeID
correct_business_type_id = 12345
update_result = db['establishments'].update_one(
    {"BusinessName": "Penang Flavours"},  # Search condition (based on BusinessName)
    {"$set": {"BusinessTypeID": correct_business_type_id}})

In [110]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Restaurant updated successfully with the correct BusinessTypeID.")
else:
    print("No changes made to the restaurant.")

No changes made to the restaurant.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [111]:
# Find how many documents have LocalAuthorityName as "Dover"
count = db['establishments'].count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 0


In [112]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db['establishments'].delete_many({"LocalAuthorityName": "Dover"})

# Print the number of deleted documents
print(f"Number of documents deleted: {delete_result.deleted_count}")

Number of documents deleted: 0


In [113]:
# Check if any remaining documents include Dover
remaining_documents = db['establishments'].count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {remaining_documents}")

Number of documents with LocalAuthorityName 'Dover': 0


In [114]:
# Check that other documents remain with 'find_one'
remaining_document = db['establishments'].find_one({"LocalAuthorityName": "Dover"})
print(remaining_document)


None


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [115]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

update_result = db['establishments'].update_many(
    {"latitude": {"$type": "string"}, "longitude": {"$type": "string"}},  
    [
        {
            "$set": {
                "latitude": {"$toDecimal": "$latitude"},  # Convert latitude to decimal
                "longitude": {"$toDecimal": "$longitude"}  # Convert longitude to decimal
            }
        }
    ]
)

print(f"Number of documents updated: {update_result.modified_count}")

Number of documents updated: 0


Use `update_many` to convert `RatingValue` to integer numbers.

In [116]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [117]:
# Change the data type from String to Integer for RatingValue
update_result = db['establishments'].update_many(
    {"RatingValue": {"$type": "string"}},  
    [
        {
            "$set": {
                "RatingValue": {"$toInt": "$RatingValue"} 
            }
        }
    ]
)

# Print the number of documents updated
print(f"Number of documents updated: {update_result.modified_count}")

Number of documents updated: 0


In [118]:
# Check that the coordinates and rating value are now numbers
sample_document = db['establishments'].find_one({
    "latitude": {"$type": "double"},  # Check if latitude is a decimal (number)
    "longitude": {"$type": "double"},  # Check if longitude is a decimal (number)
    "RatingValue": {"$type": "int"}   # Check if RatingValue is an integer
})
if sample_document:
    print("Sample document with valid data types:")
    print(sample_document)
else:
    print("No document found with valid data types for latitude, longitude, and RatingValue.")

No document found with valid data types for latitude, longitude, and RatingValue.
